In [11]:
import pandas as pd
from bs4 import BeautifulSoup as BS
import requests

In [13]:
#loading the data
#Note that this ADP_URL is for PPR. If you'd like to grab the ADP for a different league format,
#head to the url, toggle to your scoring format, and copy the URL generated
ADP_URL = 'https://www.fantasypros.com/nfl/adp/ppr-overall.php'
PROJECTION_URL = 'https://raw.githubusercontent.com/fantasydatapros/2021-VOR-Model/master/data/all_compiled.csv'
df = pd.read_csv(PROJECTION_URL, index_col=0)

# removing extra characters in player names to prepare data for merging
df['Player'] = df['Player'].apply(lambda x: ' '.join(x.split()[:2]))

#removing commas from should-be float columns that are currently strings
comma_columns = ['REC_YD', 'PASS_YD', 'RUSH_YD']
for column in comma_columns:
  df[column] = df[column].apply(lambda x: x.replace(',', ''))

#converting all columns that should be floats to floats
df.iloc[:, 3:] = df.iloc[:, 3:].astype(float)
#standard scoring
df['STANDARD'] = (df['RUSH_YD'] + df['REC_YD'])*0.1 + \
(df['RUSH_TD'] + df['REC_TD'])*6 + \
+ df['PASS_YD']*0.04 + df['PASS_TD']*4 + \
(df['INTS'] + df['FL'])*-2

#standard + 0.5 * receptions
df['HALF_PPR'] = df['STANDARD'] + df['REC']*0.5

#standard + 1 * each reception
df['PPR'] = df['STANDARD'] + df['REC']

res = requests.get(ADP_URL)
soup = BS(res.content, 'html.parser')
table = soup.find('table', {'id': 'data'})

adp_df = pd.read_html(str(table))[0]
#cleaning the data.
#player name, team, and bye week were all located in the same column, and so we are splitting the column up and creating seperate columns
#for player name and team and removing bye week altogether
adp_df['Team'] = adp_df['Player Team (Bye)'].apply(lambda x: x.split()[-2])
adp_df['Player'] = adp_df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2]))
# removing extra characters in player names to prepare data for merging
adp_df['Player'] = adp_df['Player'].apply(lambda x: ' '.join(x.split()[:2]))

adp_df['POS'] = adp_df['POS'].apply(lambda x: x[:2])
adp_df = adp_df.loc[:, ['Player', 'Team', 'POS', 'AVG']]

#creating a column to rank players on their ADP
adp_df['ADP_RANK'] = adp_df['AVG'].rank(method='first')

replacement_players = {}
for _, row in adp_df[:100].iterrows():
  replacement_players[row['POS']] = row['Player']

replacement_values = pd.DataFrame({
    'Player': replacement_players.values(),
    'POS': replacement_players.keys()
}).merge(
    df, on=['Player', 'POS']
).loc[:, ['POS', 'PPR']].rename({
    'PPR': 'REPLACEMENT_VALUE'
}, axis=1)

vor_df = df.merge(replacement_values, on='POS')

vor_df['VOR'] = vor_df['PPR'] - vor_df['REPLACEMENT_VALUE']

vor_df = vor_df.sort_values(by='VOR', ascending=False).reset_index(drop=True)

vor_df.head()

,Player,Team,POS,RUSH_ATT,RUSH_YD,RUSH_TD,REC,REC_YD,REC_TD,FL,PASS_ATT,CMP,PASS_YD,PASS_TD,INTS,STANDARD,HALF_PPR,PPR,REPLACEMENT_VALUE,VOR
0,Christian McCaffrey,CAR,RB,297.3,1302.1,12.0,93.7,758.0,4.3,2.5,0.0,0.0,0.0,0.0,0.0,298.81,345.66,392.51,139.81,252.70
1,Dalvin Cook,MIN,RB,316.6,1487.5,12.9,55.0,473.5,2.0,2.9,0.0,0.0,0.0,0.0,0.0,279.70,307.20,334.70,139.81,194.89
2,Alvin Kamara,NO,RB,207.0,942.3,9.6,81.6,704.6,3.7,1.9,0.0,0.0,0.0,0.0,0.0,240.69,281.49,322.29,139.81,182.48
3,Derrick Henry,TEN,RB,343.5,1705.8,14.1,24.4,182.6,1.1,2.7,0.0,0.0,0.0,0.0,0.0,274.64,286.84,299.04,139.81,159.23
4,Travis Kelce,KC,TE,0.0,0.0,0.0,112.4,1399.8,10.3,0.8,0.0,0.0,0.0,0.0,0.0,200.18,256.38,312.58,156.44,156.14


In [14]:
vor_adp_df = vor_df.merge(adp_df).assign(
    VOR_RANK=lambda x: x.VOR.rank(ascending=False)
).assign(
    OVER_UNDER_DRAFTED = lambda x: x.ADP_RANK - x.VOR_RANK
).loc[:, ['Player', 'Team', 'POS', 'VOR', 'VOR_RANK', 'ADP_RANK', 'OVER_UNDER_DRAFTED']]

with pd.option_context('display.max_rows', 100):
  display(vor_adp_df.head(100))

,Player,Team,POS,VOR,VOR_RANK,ADP_RANK,OVER_UNDER_DRAFTED
0,Dalvin Cook,MIN,RB,194.890,1.0,2.0,1.0
1,Alvin Kamara,NO,RB,182.480,2.0,3.0,1.0
2,Derrick Henry,TEN,RB,159.230,3.0,4.0,1.0
3,Travis Kelce,KC,TE,156.140,4.0,7.0,3.0
4,Davante Adams,GB,WR,150.650,5.0,6.0,1.0
5,Austin Ekeler,LAC,RB,141.910,6.0,10.0,4.0
6,Saquon Barkley,NYG,RB,141.430,7.0,9.0,2.0
7,Ezekiel Elliott,DAL,RB,138.900,8.0,5.0,-3.0
8,Aaron Jones,GB,RB,137.200,9.0,8.0,-1.0
9,Tyreek Hill,KC,WR,135.960,10.0,13.0,3.0


In [15]:
vor_adp_df.to_csv('2021draftstats.csv')